<a href="https://colab.research.google.com/github/rachanashinde15/Predicting-bank-deposit-using-deep-learning/blob/main/ResnetCiphar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation, Dropout
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.utils import np_utils
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar100
import numpy as np
import tensorflow as tf
import os

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
batch_size = 32
epochs = 100
num_classes = 100
depth = 20
subtract_pixel_mean = True

In [5]:
model_type = 'ResNet%d' % (depth)

In [6]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169017344/169001437 [==============================] - 6s 0us/step


In [7]:
input_shape = x_train.shape[1:]
input_shape

(32, 32, 3)

In [8]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [9]:
if subtract_pixel_mean:
  x_train_mean = np.mean(x_train, axis=0)
  x_train -= x_train_mean
  x_test -= x_train_mean

In [10]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [11]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [12]:
def lr_schedule(epoch):
  lr = 1e-3
  if epoch > 180:
      lr *= 0.5e-3
  elif epoch > 160:
      lr *= 1e-3
  elif epoch > 120:
      lr *= 1e-2
  elif epoch > 80:
      lr *= 1e-1
  print('Learning rate: ', lr)
  return lr

In [13]:
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu', batch_normalization=True, conv_first=True):
  
  conv = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')

  x = inputs
  if conv_first:
      x = conv(x)
      if batch_normalization:
          x = BatchNormalization()(x)
      if activation is not None:
          x = Activation(activation)(x)
  else:
      if batch_normalization:
          x = BatchNormalization()(x)
      if activation is not None:
          x = Activation(activation)(x)
      x = conv(x)
  return x

In [14]:
def resnet_v1(input_shape, depth, num_classes=100):
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,num_filters=num_filters,strides=strides)
            y = resnet_layer(inputs=y,num_filters=num_filters,activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,num_filters=num_filters,kernel_size=1,strides=strides,activation=None,batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
            x = Dropout(rate=0.25)(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
model = resnet_v1(input_shape=input_shape, depth=depth)

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=lr_schedule(0)),metrics=['accuracy'])

Learning rate:  0.001


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
model.summary()
print(model_type)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [18]:
save_dir = os.path.join(os.getcwd(), 'saved_models_with_dropout')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [19]:
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc',verbose=1,save_best_only=True)

In [20]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [21]:
callbacks = [checkpoint, lr_scheduler]

In [ ]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test, y_test),shuffle=True,callbacks=callbacks)

Learning rate:  0.001
Epoch 1/100
1563/1563 [==============================] - 563s 359ms/step - loss: 4.0401 - accuracy: 0.0761 - val_loss: 3.9078 - val_accuracy: 0.1029 - lr: 0.0010
Learning rate:  0.001
Epoch 2/100
1563/1563 [==============================] - 548s 351ms/step - loss: 3.5334 - accuracy: 0.1506 - val_loss: 3.6843 - val_accuracy: 0.1409 - lr: 0.0010
Learning rate:  0.001
Epoch 3/100
1563/1563 [==============================] - 548s 351ms/step - loss: 3.1975 - accuracy: 0.2080 - val_loss: 3.2511 - val_accuracy: 0.2057 - lr: 0.0010
Learning rate:  0.001
Epoch 4/100
1563/1563 [==============================] - 559s 358ms/step - loss: 2.9662 - accuracy: 0.2517 - val_loss: 2.9208 - val_accuracy: 0.2557 - lr: 0.0010
Learning rate:  0.001
Epoch 5/100
1563/1563 [==============================] - 572s 366ms/step - loss: 2.7977 - accuracy: 0.2831 - val_loss: 2.9669 - val_accuracy: 0.2590 - lr: 0.0010
Learning rate:  0.001
Epoch 6/100
1563/1563 [==============================] - 5